### Neural Chat 7B (Intel)
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/markwsl/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/neural-chat-7b-v3-3.Q6_K.gguf',
    temperature=0.1,
    max_new_tokens=1024,
    context_window=8192, # 8K context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30}, # Left at 30, no indication provided of how many layers are being offloaded
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./Models/neural-chat-7b-v3-3.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = intel_neural-chat-7b-v3-3
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  ll

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template for Neural Chat 7B:

&#35;&#35;&#35; System:<br>
{system}<br>
&#35;&#35;&#35; User:<br>
{usr}<br>
&#35;&#35;&#35; Assistant:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(systemmessage, promptmessage):
    return f"### System:\n{systemmessage}\n### User:\n{promptmessage}\n###Assistant:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_10060/2949011931.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [8]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [9]:
def RunQuestion(questionText):
    systemmessage = "You are a story teller who likes to elaborate. Answer questions in a positive, helpful and interesting way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'."

    queryQuestion = modelspecific_prompt(systemmessage, questionText)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [10]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [11]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me



llama_print_timings:        load time =    2113.18 ms
llama_print_timings:      sample time =      20.97 ms /    73 runs   (    0.29 ms per token,  3481.33 tokens per second)
llama_print_timings: prompt eval time =    2720.03 ms /  1275 tokens (    2.13 ms per token,   468.75 tokens per second)
llama_print_timings:        eval time =    2561.87 ms /    72 runs   (   35.58 ms per token,    28.10 tokens per second)
llama_print_timings:       total time =    5466.68 ms /  1347 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t


llama_print_timings:        load time =    2113.18 ms
llama_print_timings:      sample time =      17.82 ms /    60 runs   (    0.30 ms per token,  3367.19 tokens per second)
llama_print_timings: prompt eval time =    1181.84 ms /  1007 tokens (    1.17 ms per token,   852.06 tokens per second)
llama_print_timings:        eval time =    2208.53 ms /    59 runs   (   37.43 ms per token,    26.71 tokens per second)
llama_print_timings:       total time =    3542.41 ms /  1066 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached him. "Thundertooth, we need a plan to divert or neutralize the meteor. Our technology can only do so much, but with your unique abilities, perhaps we can find a solution."



Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked


llama_print_timings:        load time =    2113.18 ms
llama_print_timings:      sample time =       9.63 ms /    33 runs   (    0.29 ms per token,  3425.01 tokens per second)
llama_print_timings: prompt eval time =     645.57 ms /   996 tokens (    0.65 ms per token,  1542.82 tokens per second)
llama_print_timings:        eval time =    1156.62 ms /    32 runs   (   36.14 ms per token,    27.67 tokens per second)
llama_print_timings:       total time =    1883.23 ms /  1028 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t


llama_print_timings:        load time =    2113.18 ms
llama_print_timings:      sample time =      26.35 ms /    85 runs   (    0.31 ms per token,  3226.17 tokens per second)
llama_print_timings: prompt eval time =     324.81 ms /   411 tokens (    0.79 ms per token,  1265.37 tokens per second)
llama_print_timings:        eval time =    3254.51 ms /    84 runs   (   38.74 ms per token,    25.81 tokens per second)
llama_print_timings:       total time =    3809.56 ms /   495 tokens


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t

#### 13. Output responses

In [12]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

In the story, Thundertooth is a talking dinosaur who travels through time to the future. He encounters the Thundertooth family who helps him navigate the city. They work together to save the city from a meteor threat. Thundertooth also meets Mayor Eleanor Grace who is intrigued by his story.

--------

2/4 Who was the main protagonist?

The main protagonist in this story seems to be Thundertooth, who gathers his family members to face the impending crisis of a meteor threatening their city. He plays a crucial role in coordinating the efforts to protect the city and its citizens. [1-3]

--------

3/4 Did they have any children? If so, what were their names?

Thundertooth and Seraphina had four children together. Their names were Lumina, Echo, Sapphire, and Ignis.

--------

4/4 Did anything eventful happen?

Yes, the story revolves around the Thundertooth family facing a crisis involving a meteor threatening their city. They work together to devise a plan